In [1]:
from xml.dom import minidom
import pandas as pd
import numpy as np
import os
import glob
import time
from datetime import datetime
from datetime import timedelta
import gzip
import mechanicalsoup
import urllib
import requests
from bs4 import BeautifulSoup
import json
import time
from langdetect import detect
import random

In [15]:
def get_wos_data(file, year):
    soup = BeautifulSoup(open(file, encoding='utf-8'), 'html.parser')
    
    # Title
    title = str(soup.find("div", {"class": "title"}))
    title = title[title.find('<value>') + 7:title.find('</value>')]
    
    # Authors
    authors = []
    authors_array = soup.findAll("a", {"title": "Find more records by this author"})

    for author in authors_array:
        author = str(author)
        authors.append(author[author.rfind('author')+8:author.rfind('</a>')] + '.')
    
    # Keywords
    a_array = soup.findAll("a", {"class": "snowplow-author-keyword-link"})
    keywords = []

    for a in a_array:
        a = str(a)
        keywords.append(a[a.rfind('keywords') + 10:-4])
    
    # Abstract
    p_array = soup.findAll("p", {"class": "FR_field"})
    abstract = ''
    for p in p_array:
        p = str(p)
        if p[19] == '>' and (p[20] != '\n' and p[20] != ' '):
            abstract = p[p.find('>') + 1:-4]
    
    # No. of citations
    h2 = str(soup.find('h2'))
    citations = h2[h2.find(':') + 2:-5]
    
    # Funding agency
    tr = soup.find("tr", {"class": "fr_data_row"})
    funding = ''
    if tr is not None:
        funding = tr.find("td").string[:-1]
        
    # Language
    lang = ''
    if len(abstract):
        lang = detect(abstract)
    
    data = {}
    data['Title'] = title
    data['Authors'] = authors
    data['Keywords'] = keywords
    data['Date'] = year
    data['Abstract'] = abstract
    data['Language'] = lang
    data['Citations'] = citations
    data['Funding_agency'] = funding
    
    json_data = json.dumps(data)
    return json_data


def get_newest_file(path):
    files = os.listdir(path)
    paths = [os.path.join(path, basename) for basename in files]
    return max(paths, key=os.path.getctime)

Iz XML datoteke, ki smo jo dobili iz Sicrisa, izvlečemo DOI člankov, ki ga bomo uporabili za iskanje člankov po portalu Web of Scence

In [20]:
xmldoc = minidom.parse('downloads/cobiss-org/2019-08-26-153312_cobiss.xml')
articles = xmldoc.getElementsByTagName('BiblioEntry')
WoSData = {}
for article in articles:
    wos = article.getElementsByTagName('WoS')
    date = article.getElementsByTagName('PubDate')
    
    if (len(wos) != 0 and len(date) != 0):
        articleId = wos[0].childNodes[0].nodeValue[-15:]
        
        for d in date:
            if len(d.childNodes):
                year = d.childNodes[0].nodeValue
                break
                
        year = year.strip('cop. ')
        year = year[:4]
        WoSData[articleId] = year
        
articleIds = list(WoSData.keys())

In [21]:
browser = mechanicalsoup.StatefulBrowser(user_agent='Mozilla/5.0 (X11; Linux i686; rv:10.0) Gecko/20100101 Firefox/10.0')
now = datetime.now()

count = 1

for index in articleIds:
    if not os.path.exists('downloads/wos/{:}'.format(index)):
        os.mkdir('downloads/wos/{:}'.format(index))

    if len(os.listdir('downloads/wos/{:}'.format(index))) != 0:
        file = get_newest_file('downloads/wos/{:}'.format(index))
        file_date = datetime.strptime(file[file.rfind('/')+1:-5], '%Y-%m-%d')
        if file_date < now - timedelta(days = 14):
            
            browser.open('http://gateway.isiknowledge.com/gateway/Gateway.cgi?GWVersion=2&SrcAuth=Alerting&SrcApp=Alerting&DestApp=WOS&DestLinkType=FullRecord&UT={:}'.format(index))

            file = open('downloads/wos/{:}/{:}.html'.format(index, now.strftime('%Y-%m-%d')), "w", encoding='utf-8')
            file.write(str(browser.get_current_page()))
            file.close()
            
            time.sleep(random.randint(3, 5))
            count += 1

    else:
        browser.open('http://gateway.isiknowledge.com/gateway/Gateway.cgi?GWVersion=2&SrcAuth=Alerting&SrcApp=Alerting&DestApp=WOS&DestLinkType=FullRecord&UT={:}'.format(index))
        file = open('downloads/wos/{:}/{:}.html'.format(index, now.strftime('%Y-%m-%d')), "w", encoding='utf-8')
        file.write(str(browser.get_current_page()))
        file.close()
        
        time.sleep(random.randint(3, 5))
        count += 1
        
    if count % 200 == 0:
        time.sleep(600)
        print('New pages saved: {:}'.format(count))
        count += 1

browser.close()
print('Total new files: {:}'.format(count - 1))

Total new files: 0


In [23]:
directories = os.listdir('./downloads/wos')
file_data = []

for directory in directories:
    if directory[0] == '.':
        continue
    
    if len(os.listdir('downloads/wos/{:}'.format(directory))) != 0:
        file = get_newest_file('./downloads/wos/{:}'.format(directory))
    
        file_data.append(get_wos_data(file, WoSData[directory]))
        
file_data;

In [24]:
tstamp = datetime.now().strftime('%Y-%m-%d-%H%M%S')
file_name = 'data/wos_data_{:}.json'.format(tstamp)

with open(file_name, 'w') as f:
    f.write('[\n')
    for item in file_data:
        if (item == file_data[len(file_data) - 1]):
            f.write("%s\n" % item)
        else:
            f.write("%s,\n" % item)
    f.write(']')